In [1]:
import pandas as pd
import numpy as np

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
)

In [2]:
pl.seed_everything(42)

Global seed set to 42


42

In [3]:
MODEL_NAME = 't5-small'
LEARNING_RATE = 0.0001

In [4]:
SEP_TOKEN = '<sep>'
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
print('tokenizer len before: ', len(tokenizer))
tokenizer.add_tokens(SEP_TOKEN)
print('tokenizer len after: ', len(tokenizer))
TOKENIZER_LEN = len(tokenizer)

tokenizer len before:  32100
tokenizer len after:  32101


In [5]:
class QGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.model.resize_token_embeddings(TOKENIZER_LEN) #resizing after adding new tokens to the tokenizer

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=LEARNING_RATE)

In [10]:
print(torch.__version__)

2.0.1


In [26]:
checkpoint_path = "QA Generation Using T5 and SQuAD/best-checkpoint-v4.ckpt"
QGmodel = QGModel.load_from_checkpoint(checkpoint_path, map_location=torch.device('cpu'))
QGmodel.freeze()
QGmodel.eval()

QGModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32101, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32101, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Linear(in_features=204

In [27]:
checkpoint_path = "Distractor Generation using T5 and RACE/best-checkpoint-v16.ckpt"
DstrModel = QGModel.load_from_checkpoint(checkpoint_path, map_location=torch.device('cpu'))
DstrModel.freeze()
DstrModel.eval()

QGModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(32101, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32101, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=512, bias=False)
                (k): Linear(in_features=512, out_features=512, bias=False)
                (v): Linear(in_features=512, out_features=512, bias=False)
                (o): Linear(in_features=512, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=512, out_features=2048, bias=False)
                (wo): Linear(in_features=204

In [28]:
def generateQuestion(qgmodel: QGModel, answer: str, context: str) -> str:
    source_encoding = tokenizer(
        '{} {} {}'.format(answer, SEP_TOKEN, context),
        max_length=300,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    generated_ids = qgmodel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=1,
        max_length=80,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)

In [29]:
def generateOptions(qgmodel: QGModel, answer: str, question: str, context: str) -> str:
    source_encoding = tokenizer(
        '{} {} {} {} {}'.format(answer, SEP_TOKEN, question, SEP_TOKEN, context),
        max_length=512,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )

    generated_ids = qgmodel.model.generate(
        input_ids=source_encoding['input_ids'],
        attention_mask=source_encoding['attention_mask'],
        num_beams=1,
        max_length=64,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    preds = {
        tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    }

    return ''.join(preds)

In [30]:
df_test = pd.read_csv("Datasets/RACE/race_test_df.csv")

In [31]:
df_test.head()

,context,question,correct,incorrect1,incorrect2,incorrect3
0,The rain had continued for a week and the floo...,What did Nancy try to do before she fell over?,Protect her cows from being drowned,Measure the depth of the river,Look for a fallen tree trunk,Run away from the flooded farm
1,The rain had continued for a week and the floo...,The following are true according to the passag...,Nancy took hold of the rope and climbed into t...,It took Lizzie and Nancy about 20 minutes to g...,It was raining harder when Nancy managed to ge...,The bad weather made it difficult for rescuers...
2,The rain had continued for a week and the floo...,What did the local people do to help those in ...,They put up shelter for them in a school.,They used helicopters to help carry cows.,They helped farmers gather their cows.,They set up an organization called Red Cross.
3,There is probably no field of human activity i...,The passage tells us that _ .,the clothes that we choose to wear have someth...,our values and lifestyles are in no field of h...,our values and lifestyles are from the sign la...,the clothes we choose to wear depend on a set ...
4,There is probably no field of human activity i...,"Traditionally,people usually thought that _ .",women were concerned greatly about what they w...,men cared very much for clothes,both men and women paid great attention to the...,neither men nor women showed interest in clothes


In [32]:
df_test['context'][0]

'The rain had continued for a week and the flood had created a big river which were running by Nancy Brown\'s farm. As she tried to gather her cows to a higher ground, she slipped and hit her head on a fallen tree trunk. The fall made her unconscious for a moment or two. When she came to, Lizzie, one of her oldest and favorite cows, was licking her face. \nAt that time, the water level on the farm was still rising. Nancy gathered all her strength to get up and began walking slowly with Lizzie. The rain had become much heavier, and the water in the field was now waist high. Nancy\'s pace got slower and slower because she felt a great pain in her head. Finally, all she could do was to throw her arm around Lizzie\'s neck and try to hang on. About 20 minutes later, Lizzie managed to pull herself and Nancy out of the rising water and onto a bit of high land, which seemed like a small island in the middle of a lake of white water. \nEven though it was about noon, the sky was so dark and the 

In [33]:
generateQuestion(QGmodel, df_test['correct'][0], df_test['context'][0])

'<pad> Protect her cows from being drowned<sep> What was the main reason Lizzie fell on a tree trunk?</s>'

In [34]:
generateOptions(DstrModel, df_test['correct'][0], df_test['question'][0], df_test['context'][0])

'<pad> Get up and walk slowly with Lizzie<sep> Keep her feet licking herself in the field.<extra_id_29> Get up to get up quickly</s>'

In [35]:
def generate(answer, context):
    ques = generateQuestion(QGmodel, answer, context)
    # Extracting question
    flag = 0
    question = []
    for word in ques.split():
        if '</s>' in word:
            word = word.replace('</s>', '')
        if flag == 1:
            question.append(word)
        if '<sep>' in word:
            flag = 1
    question = ' '.join(question)
    optns = generateOptions(DstrModel, answer, question, context)
    # Extracting options
    flag = 0
    options = []
    temp = []
    optns = optns.replace('<', ' <')
    for word in optns.split():
        if word == '<pad>':
            continue
        if word == '<sep>':
            options.append(' '.join(temp))
            temp = []
            continue
        if '<extra_id' in word:
            options.append(' '.join(temp))
            temp = []
            continue
        if word == '</s>':
            options.append(' '.join(temp))
            temp = []
            continue
        temp.append(word)
    # options = {'correct': answer, 'incorrect': options}
    final_text = question + '\n'
    opt = {'a': answer}
    i = 1
    for o in options:
        opt[chr(97+i)] = o
        i = i+1
    final_text = 'Question: ' + question
    for keys in opt.keys():
        final_text = final_text + '\n' + keys + '. ' + opt[keys]
    final_text = final_text + '\n\nCorrect Options is a.'
    return final_text

In [36]:
df_test['correct'][2]

'They put up shelter for them in a school.'

In [37]:
print(generate(df_test['correct'][2], df_test['context'][2]))

Question: What did the rescuers do to find Nancy?
a. They put up shelter for them in a school.
b. They lowered a rope.
c. They lowered ice to catch her in the water and then put them into dangers on their farm, too!
d. They landed on the island from frightened helicopter

Correct Options is a.


In [39]:
import gradio as gr
interface = gr.Interface(
    fn=generate,
    inputs=["text", "text"],
    outputs="text",
    title="Generate Quiz",
    description="Generate mcqs for the given answer and context using T5 model.",
    examples=[
        [df_test['correct'][2], df_test['context'][2]],
    ]
)

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Running on public URL: https://dfe319d6a959a9bce7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### Getting results

In [40]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 783.9 kB/s eta 0:00:00a 0:00:01
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 3.8 MB/s eta 0:00:0000:0100:01


In [42]:
results = []
for i in range(len(df_test)):
    results.append(generate(df_test['correct'][i], df_test['context'][i]))

KeyboardInterrupt: 

In [43]:
len(results)

4608

In [49]:
print(results[4])

Question: What was the reason for the concern of clothes?
a. women were concerned greatly about what they wore while men didn't
b. Women were not aware of what they wore while men didn't
c. Men were not aware about what they wore when women did.
d. Women were too proud to wear clothes

Correct Options is a.


In [50]:
file = open('items.txt','w')
for res in results:
    file.write(res+"\n")
file.close()